# GOALS
To create a GBQ view containing a daily state on users engagement.
- generate dates index
- generate users and dates index
- fill last_login_date for each date/user key
- fill last_consumption_start_date for each date/user key
- fill last_consumption_completion_date for each date/user key

# PACKAGES

In [76]:
import pandas as pd
from google.oauth2 import service_account
import pandas_gbq
import logging

# PARAMETERS

In [77]:
project_id = "analytics-dev-308300"

cred_file="../keys/gcp_key.json"

destination_table='raw_engagement.users_engagement'

In [78]:
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

# FUNCTIONS

## get data

In [79]:
def get_data(project_id, credentials):
    """
    ()-->df
    """
    query = """
        SELECT DISTINCT
            user_id,
            created_at,
            extraction_date,
            user_name
        FROM
            dtm_engagement.dim_users
        WHERE
            group_id=1818
    """
    users_df = pd.read_gbq(
        query=query, credentials=credentials, project_id=project_id)

    users_lst = users_df['user_id'].tolist()

    
    creation_df= users_df[['user_id','user_name','created_at']].drop_duplicates(ignore_index=True)
    creation_df['created_at']=pd.to_datetime(
        creation_df['created_at'], utc=True
        )

    query = """
        SELECT DISTINCT
            user_id,
            days_since_last_login,
            extraction_date,
            joined_group
        FROM
            dtm_engagement.hist_users
        WHERE
            group_id=1818
        ORDER BY
            extraction_date DESC
        """

    login_df = pd.read_gbq(
        query=query, credentials=credentials, project_id=project_id)

    login_df=login_df.drop_duplicates(subset=['user_id','extraction_date'],keep='last',ignore_index=True)
    
#     login_df["last_login"] = pd.to_datetime(
#         login_df["last_login"], utc=True
#     )

    query = """
        SELECT DISTINCT
            user_id,
            set_id,
            started_at,
            completed_at
        FROM
            dtm_engagement.ft_content_consumption
        WHERE
            group_id=1818
        """
    consumption_df = pd.read_gbq(
        query=query, credentials=credentials, project_id=project_id
    )

    consumption_df["started_at"] = pd.to_datetime(
        consumption_df["started_at"], utc=True
    )
    consumption_df["completed_at"] = pd.to_datetime(
        consumption_df["completed_at"], utc=True
    )

    return users_lst, creation_df, login_df, consumption_df

## generate base (users and dates) data frame

In [80]:
def create_base_df(
    users_ls,
    creation_df,
    start_date="2021-08-16",
    end_date=pd.Timestamp.today().strftime("%Y-%m-%d"),
):
    """
    (date-like, date_like, series) --> df
    Create a dataframe with one row for each combination of user and date. Date range is defined by start_date and end_date (excluded).
    """
    dates_index = (
        pd.to_datetime(
            pd.date_range(start=start_date, end=end_date, name="action_date")
        )
        .strftime("%Y-%m-%d")
        .to_list()
    )

    actions_dict = [
        {"action_date": action_date, "user_id": user}
        for action_date in dates_index
        for user in users_ls
    ]
    
    base_df=pd.DataFrame(actions_dict)
    base_df=base_df.merge(creation_df, how='left',on='user_id')
    base_df=base_df.drop(index=base_df[base_df['created_at']>base_df['action_date']].index)
    
    return base_df

## fill max date of interest for each date/user key

In [81]:
def max_date(consumption_df, reporting_date, user_id, date_of_interest):
    """
    (df,date like str, int)-->date
    Select the maximum value for date_of_interest field that is inferior to the reporting date (23:59:59), for the specified user_id.
    """
    max_start = consumption_df[
        (consumption_df["user_id"] == user_id)
        & (
            consumption_df[date_of_interest]
            <= pd.Timestamp(reporting_date + " 23:59:59", tz="UTC")
        )
    ][date_of_interest].max()

    return max_start

## Fill the number of completed sets by period

In [82]:
def calculate_nb_of_sets_of_interest(consumption_df,reporting_date,user_id,date_of_interest,nb_of_days):
    """
    (df, date like str, int, int, date like str)-->int
    For the user_id, count the number of set_ids where completed at is between reporting_date-number_of_days and reporting_date.
    """

    number_of_sets = consumption_df[(consumption_df['user_id']==user_id) & 
                                    (consumption_df[date_of_interest].between(pd.Timestamp(reporting_date + " 23:59:59", tz="UTC")-pd.Timedelta(nb_of_days,'days'),pd.Timestamp(reporting_date + " 23:59:59", tz="UTC")))]['set_id'].count()
    
    return number_of_sets

## Set user status based on dates

In [83]:
def user_status(days_since_last_login, days_since_last_start,days_since_last_completion,ever_logged):
    """
    (timedelta,timedelta,timedelta,bool)--> str
    """
    if ever_logged==False:
        return '0.bird'
    elif days_since_last_completion <= 7:
        return '4.learner'
    elif days_since_last_start <= 7:
        return '3.consumer'
    elif days_since_last_login <= 7:
        return '2.curious'
    else:
        return '1.missing'

# assert user_status(
#     timedelta_since_last_login=pd.Timedelta(pd.NaT),
#     timedelta_since_last_start=pd.Timedelta('2 days 13:41:36'),
#     timedelta_since_last_completion=pd.Timedelta('1 days 11:29:23'),
#     ever_logged=True)=='4.learner'

# assert user_status(
#     timedelta_since_last_login=pd.Timedelta(pd.NaT),
#     timedelta_since_last_start=pd.Timedelta('1 days 13:41:36'),
#     timedelta_since_last_completion=pd.Timedelta('7 days 11:29:23'),
#     ever_logged=True)=='3.consumer'

# assert user_status(
#     timedelta_since_last_login=pd.Timedelta(pd.NaT),
#     timedelta_since_last_start=pd.Timedelta('1 days 13:41:36'),
#     timedelta_since_last_completion=pd.Timedelta(pd.NaT),
#     ever_logged=True)=='3.consumer'

# # assert user_status(
# #     timedelta_since_last_login=pd.Timedelta('1 days 13:41:36'),
# #     timedelta_since_last_start=pd.Timedelta('8 days 13:41:36'),
# #     timedelta_since_last_completion=pd.Timedelta('7 days 11:29:23'),
# #     ever_logged=True)=='2.curious'

# assert user_status(
#     timedelta_since_last_login=pd.Timedelta('7 days 13:41:36'),
#     timedelta_since_last_start=pd.Timedelta('8 days 13:41:36'),
#     timedelta_since_last_completion=pd.Timedelta('10 days 11:29:23'),
#     ever_logged=True)=='1.missing'

# assert user_status(
#     timedelta_since_last_login=pd.Timedelta('0 days'),
#     timedelta_since_last_start=pd.Timedelta('0 days'),
#     timedelta_since_last_completion=pd.Timedelta('0 days'),
#     ever_logged=False)=='0.bird'

## create engagement df by completing base_df with calculated fields

In [84]:
def generate_engagement_df(base_df,consumption_df,login_df):
    """
    (df,df,df)-->df
    """
    engagement_df=base_df.copy()
    
    engagement_df=pd.merge(engagement_df,login_df,left_on=['user_id','action_date'],right_on=['user_id','extraction_date'])
    
    engagement_df['last_start_date']=engagement_df.apply(lambda x: max_date(consumption_df,x['action_date'],x['user_id'],'started_at'), axis=1)
    engagement_df['timedelta_since_last_start']=pd.to_datetime(engagement_df['action_date']+' 23:59:59',utc=True)-engagement_df['last_start_date']
    engagement_df['days_since_last_start']=engagement_df['timedelta_since_last_start'].dt.days
    
    engagement_df['last_completion_date']=engagement_df.apply(lambda x: max_date(consumption_df,x['action_date'],x['user_id'],'completed_at'), axis=1)
    engagement_df['timedelta_since_last_completion']=(
        pd.to_datetime(engagement_df['action_date']+' 23:59:59',utc=True)-engagement_df['last_completion_date']
            )
    engagement_df['days_since_last_completion']=engagement_df['timedelta_since_last_completion'].dt.days
    
    engagement_df['nb_of_completed_sets']=engagement_df.apply(lambda x: calculate_nb_of_sets_of_interest(consumption_df=consumption_df,reporting_date=x['action_date'],user_id=x['user_id'],date_of_interest='completed_at',nb_of_days=7),axis=1)
    
    engagement_df['user_status']=engagement_df.apply(lambda x:
                                                               user_status(
                                                                   x['days_since_last_login'],
                                                                   x['days_since_last_start'],
                                                                   x['days_since_last_completion'],
                                                                   ever_logged=x['joined_group']),
                                                               axis=1
                                                              )
    
    return engagement_df

## create user_engagement table in gbq

In [85]:
def create_engagement_table(project_id=project_id, destination_table_name=destination_table, cred_file=cred_file):
    """
    (df,str,str,str)--> gbq table
    """
    
    credentials = service_account.Credentials.from_service_account_file(cred_file)
    
    users_ls, creation_df, login_df, consumption_df = get_data(project_id=project_id,credentials=credentials)
    
    engagement_df = generate_engagement_df(
                                create_base_df(users_ls=users_ls,
                                               creation_df=creation_df),
                                consumption_df=consumption_df,
                                login_df=login_df
                    )
    
    engagement_df.to_gbq(destination_table_name,project_id=project_id,credentials=credentials,if_exists='replace')
    
    return engagement_df

# DATA WRANGLING

In [88]:
create_engagement_table()

1it [00:03,  3.21s/it]


,action_date,user_id,user_name,created_at,days_since_last_login,extraction_date,joined_group,last_start_date,timedelta_since_last_start,days_since_last_start,last_completion_date,timedelta_since_last_completion,days_since_last_completion,nb_of_completed_sets,user_status
0,2021-08-23,20029,JM Benedetto,2021-07-25 00:11:04+00:00,NaN,2021-08-23,None,NaT,NaT,NaN,NaT,NaT,NaN,0,1.missing
1,2021-08-23,20030,Aurelien Jacomy,2021-07-25 02:42:03+00:00,NaN,2021-08-23,None,NaT,NaT,NaN,NaT,NaT,NaN,0,1.missing
2,2021-08-23,20032,Martin Ciriani,2021-07-25 21:00:47+00:00,NaN,2021-08-23,None,NaT,NaT,NaN,NaT,NaT,NaN,0,1.missing
3,2021-08-23,20082,H Aluno,2021-07-31 23:54:33+00:00,NaN,2021-08-23,None,NaT,NaT,NaN,NaT,NaT,NaN,0,1.missing
4,2021-08-23,20100,Luana Amarante,2021-08-03 11:05:51+00:00,NaN,2021-08-23,None,NaT,NaT,NaN,NaT,NaT,NaN,0,1.missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,2021-09-06,20165,Jose Carlos Pasquini Catozich,2021-08-12 11:33:06+00:00,1.0,2021-09-06,True,2021-08-30 16:25:35+00:00,7 days 07:34:24,7.0,2021-09-04 21:58:58+00:00,2 days 02:01:01,2.0,1,4.learner
629,2021-09-06,20118,Martin Ciriani,2021-08-06 12:29:27+00:00,3.0,2021-09-06,True,NaT,NaT,NaN,NaT,NaT,NaN,0,2.curious
630,2021-09-06,20293,Ricardo CRKS,2021-08-25 13:15:49+00:00,2.0,2021-09-06,True,2021-08-30 20:08:42+00:00,7 days 03:51:17,7.0,2021-09-02 20:29:00+00:00,4 days 03:30:59,4.0,1,4.learner
631,2021-09-06,20138,Sergio Silva,2021-08-09 17:03:28+00:00,2.0,2021-09-06,True,2021-09-03 19:48:04+00:00,3 days 04:11:55,3.0,2021-09-03 19:50:02+00:00,3 days 04:09:57,3.0,3,4.learner


In [ ]:
credentials = service_account.Credentials.from_service_account_file(cred_file)
a,b,c,d = get_data(project_id=project_id, credentials=credentials)

In [ ]:
c['timedelta_since_last_login'].to_datetime()